In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, Model
from tensorflow.keras.models import Model
import albumentations as A
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.losses import MeanSquaredError
import tensorflow.keras.backend as K
import numpy as np

In [ ]:


# Function to compute SSIM and KL Divergence based loss
def weighted_vae_loss(y_true, y_pred, mu, log_var, weight_ssim=0.8):
    mse_loss = MeanSquaredError()(y_true, y_pred)
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    kl_loss = -0.5 * tf.reduce_mean(1 + log_var - K.square(mu) - K.exp(log_var))
    loss = weight_ssim * ssim_loss + (1 - weight_ssim) * mse_loss + kl_loss
    return loss

# VAE model definition
def build_vae(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 256x256
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 128x128
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 64x64

    x = layers.Flatten()(x)
    mu = layers.Dense(128)(x)
    log_var = layers.Dense(128)(x)

    def sampling(args):
        mu, log_var = args
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mu + tf.exp(0.5 * log_var) * epsilon

    z = layers.Lambda(sampling)([mu, log_var])

    # Decoder
    decoder_input = layers.Input(shape=(128,))
    x = layers.Dense(64 * 64 * 128, activation='relu')(decoder_input)
    x = layers.Reshape((64, 64, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 128x128
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 256x256
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 512x512

    outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

    encoder = Model(inputs, [z, mu, log_var], name='encoder')
    decoder = Model(decoder_input, outputs, name='decoder')

    vae_outputs = decoder(encoder(inputs)[0])
    vae = Model(inputs, vae_outputs, name='vae')

    vae.add_loss(weighted_vae_loss(inputs, vae_outputs, encoder(inputs)[1], encoder(inputs)[2]))
    vae.compile(optimizer='adam')

    return vae, encoder, decoder

# Data augmentation using albumentations
def augment_image(image):
    transform = A.Compose([
        A.CLAHE(p=1.0),
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ElasticTransform(p=0.5),
    ])
    augmented = transform(image=image)
    return augmented['image']

# Simulate training data (assuming a single 512x512 clean brain image)
img_path = '61_processed_image_edit.png'
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # Assuming grayscale MRI image
image = cv2.resize(image, (512, 512))  # Resize to (512, 512)

# Albumentations requires the input to have 3 channels, so expand dimensions
image = np.expand_dims(image, axis=-1)
image = image / 255.0  # Normalize

# Build VAE model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))

# Train the VAE
epochs = 5000
batch_size = 32

for epoch in range(epochs+1):
    # Perform data augmentation on each epoch
    augmented_image = augment_image(image)
    augmented_image = np.expand_dims(augmented_image, axis=0)  # Add batch dimension

    # Train the model
    vae.fit(augmented_image, augmented_image, epochs=1, batch_size=batch_size, verbose=1)

    # Print and display outputs every 50 epochs
    if epoch % 50 == 0:
        reconstructed = vae.predict(np.expand_dims(image, axis=0))
        reconstructed = np.squeeze(reconstructed)  # Remove the batch dimension

        print(f"Epoch: {epoch}")
        
        # Display original, augmented, and reconstructed images
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))

        ax[0].imshow(np.squeeze(image), cmap='gray')
        ax[0].set_title("Original Image")
        ax[0].axis("off")

        ax[1].imshow(np.squeeze(augmented_image), cmap='gray')
        ax[1].set_title("Augmented Image")
        ax[1].axis("off")

        ax[2].imshow(reconstructed, cmap='gray')
        ax[2].set_title("Reconstructed Image")
        ax[2].axis("off")

        plt.show()

    # Save model every 100 epochs
    if epoch % 100 == 0:
        vae.save(f"vae_model_epoch_{epoch}.h5")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Define the same VAE architecture used for saving the model
def build_vae(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 256x256
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 128x128
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 64x64

    x = layers.Flatten()(x)
    mu = layers.Dense(128)(x)
    log_var = layers.Dense(128)(x)

    def sampling(args):
        mu, log_var = args
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mu + tf.exp(0.5 * log_var) * epsilon

    z = layers.Lambda(sampling)([mu, log_var])

    # Decoder
    decoder_input = layers.Input(shape=(128,))
    x = layers.Dense(64 * 64 * 128, activation='relu')(decoder_input)
    x = layers.Reshape((64, 64, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 128x128
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 256x256
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 512x512

    outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

    encoder = Model(inputs, [z, mu, log_var], name='encoder')
    decoder = Model(decoder_input, outputs, name='decoder')

    vae_outputs = decoder(encoder(inputs)[0])
    vae = Model(inputs, vae_outputs, name='vae')

    return vae, encoder, decoder

# Load the model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))
vae.load_weights("vae_model_epoch_10000.h5")  # Replace with the correct model path


In [ ]:
# Load and preprocess the anomalous image in the same way as the clean image

anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)


# Predict and compute anomaly
reconstructed_anomalous = vae.predict(anomalous_image)
difference = np.abs(anomalous_image - reconstructed_anomalous)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.subplot(1, 3, 2)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.subplot(1, 3, 3)
plt.title("Tumor Region (Difference)")
plt.imshow(np.squeeze(difference), cmap='hot')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load and preprocess the anomalous image in the same way as the clean image
anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)

# Predict and compute anomaly
reconstructed_anomalous = vae.predict(anomalous_image)
difference = np.abs(anomalous_image - reconstructed_anomalous)

# Thresholding to create a binary mask
threshold_value = 0.1  # Set a threshold value to identify anomaly
binary_mask = difference > threshold_value  # Binary mask (True where difference > threshold)

# Convert the mask to float for visualization purposes
binary_mask = binary_mask.astype(np.float32)

# Plot the anomalous image, reconstructed clean image, and the tumor region (binary mask)
plt.figure(figsize=(12, 4))

# Anomalous Image
plt.subplot(1, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.axis('off')

# Reconstructed Clean Image
plt.subplot(1, 3, 2)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.axis('off')

# Tumor Region (Binary Mask)
plt.subplot(1, 3, 3)
plt.title("Tumor Region (Binary Mask)")
plt.imshow(np.squeeze(binary_mask), cmap='hot')
plt.axis('off')

plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Load and preprocess the anomalous image in the same way as the clean image
anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)

# Pass the image through the encoder to get the latent space (z, mu, log_var)
z, mu, log_var = encoder.predict(anomalous_image)

# Decode the latent representation (z) to reconstruct the clean image
reconstructed_anomalous = decoder.predict(z)

# Compute the difference between the original anomalous image and the reconstructed image
difference = np.abs(anomalous_image - reconstructed_anomalous)

# Thresholding to create a binary mask for the tumor region
threshold_value = 0.2  # You can adjust this value
binary_mask = difference > threshold_value
binary_mask = binary_mask.astype(np.float32)

# Visualize the original anomalous image, reconstructed clean image, and the tumor region (binary mask)
plt.figure(figsize=(12, 6))

# Anomalous Image
plt.subplot(2, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.axis('off')

# Latent Space (mu)
plt.subplot(2, 3, 2)
plt.title("Latent Space (mu)")
plt.plot(mu[0])  # Plot the latent space (mu) for visualization
plt.axis('on')

# Latent Space (log_var)
plt.subplot(2, 3, 3)
plt.title("Latent Space (log_var)")
plt.plot(log_var[0])  # Plot the latent space (log_var) for visualization
plt.axis('on')

# Reconstructed Clean Image
plt.subplot(2, 3, 4)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.axis('off')

# Tumor Region (Difference)
plt.subplot(2, 3, 5)
plt.title("Tumor Region (Difference)")
plt.imshow(np.squeeze(difference), cmap='hot')
plt.axis('off')

# Tumor Region (Binary Mask)
plt.subplot(2, 3, 6)
plt.title("Tumor Region (Binary Mask)")
plt.imshow(np.squeeze(binary_mask), cmap='hot')
plt.axis('off')

plt.tight_layout()
plt.show()

# Optionally, return the latent space vectors for further analysis
print("Latent space (mu):", mu[0])
print("Latent space (log_var):", log_var[0])


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

# Function to calculate reconstruction error
def calculate_reconstruction_error(original, reconstructed):
    return np.mean(np.square(original - reconstructed), axis=(1, 2, 3))  # MSE error per image

# Function to classify as anomalous based on a threshold
def classify_anomalies(reconstruction_error, threshold):
    return reconstruction_error > threshold

# Load test images (assuming grayscale 512x512 MRI images)
def load_image(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (512, 512))
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = image / 255.0  # Normalize
    return np.expand_dims(image, axis=0)  # Add batch dimension

# Define a function to evaluate model performance and show reconstructed images
def evaluate_model_and_display(vae, test_images, true_labels, threshold):
    reconstruction_errors = []
    predictions = []
    
    for img in test_images:
        original = img
        reconstructed = vae.predict(img)  # Predict reconstructed image
        error = calculate_reconstruction_error(original, reconstructed)
        reconstruction_errors.append(error)
        predictions.append(classify_anomalies(error, threshold))
    
        # Display original and reconstructed images
        plt.figure(figsize=(10, 5))
        
        # Original image
        plt.subplot(1, 2, 1)
        plt.imshow(np.squeeze(original), cmap='gray')
        plt.title("Original Image")
        plt.axis("off")

        # Reconstructed image
        plt.subplot(1, 2, 2)
        plt.imshow(np.squeeze(reconstructed), cmap='gray')
        if classify_anomalies(error, threshold):
            plt.title("Reconstructed Image (Anomalous)")
        else:
            plt.title("Reconstructed Image (Normal)")
        plt.axis("off")
        
        plt.show()
    
    reconstruction_errors = np.array(reconstruction_errors)
    predictions = np.array(predictions)

    # Calculate evaluation metrics
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    roc_auc = roc_auc_score(true_labels, reconstruction_errors)

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"ROC-AUC: {roc_auc}")

    return predictions, reconstruction_errors

# Example usage
test_image_paths = ['test/processed_image_edit.png',r"C:\Users\priya\Documents\DL project\test 2\processed_image_edit.png",r"C:\Users\priya\Documents\DL project\test 3\processed_image_edit.png"]  # Replace with actual image paths
test_images = [load_image(path) for path in test_image_paths]
true_labels = [1, 0, 0]  # Ground truth labels (0 = normal, 1 = anomalous)

# Load the saved VAE model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))
vae.load_weights("vae_model_epoch_9900.h5")  # Replace with the correct path

# Set an anomaly threshold
anomaly_threshold = 0.05  # You can tune this based on your validation results

# Evaluate the model and display reconstructed images
predictions, reconstruction_errors = evaluate_model_and_display(vae, test_images, true_labels, anomaly_threshold)

# Plot the reconstruction errors
plt.figure(figsize=(10, 5))
plt.plot(reconstruction_errors, label="Reconstruction Errors")
plt.axhline(y=anomaly_threshold, color='r', linestyle='--', label="Anomaly Threshold")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.metrics import structural_similarity as ssim

# Function to load an image
def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    image = cv2.resize(image, (512, 512))  # Resize to the required dimensions
    return image / 255.0  # Normalize to [0, 1]

# Function to compute SSIM map and score
def compute_ssim_map(original, reconstructed):
    ssim_map = ssim(original, reconstructed, full=True, data_range=1.0)[1]
    ssim_score = np.mean(ssim_map)  # Average SSIM score
    return ssim_map, ssim_score

# Function to highlight anomalous regions based on SSIM
def highlight_anomalous_regions(ssim_map, threshold):
    # Create the anomaly mask
    anomaly_mask = ssim_map < threshold
    return anomaly_mask.astype(float)  # Ensure the mask is in float format for visualization

# Function to visualize original, reconstructed, and anomaly mask
def visualize_anomalous_regions(original, reconstructed, anomaly_mask):
    plt.figure(figsize=(15, 5))

    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original Image")
    plt.axis("off")

    # Anomaly mask
    plt.subplot(1, 3, 2)
    plt.imshow(anomaly_mask, cmap='hot', alpha=0.6)  # Display anomalous regions using heatmap
    plt.title("Anomalous Regions (SSIM < Threshold)")
    plt.axis("off")

    # Reconstructed image
    plt.subplot(1, 3, 3)
    plt.imshow(reconstructed, cmap='gray')
    plt.title("Reconstructed Image")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# Function to evaluate model and display anomalies
def evaluate_and_display_anomalies(vae, test_images, threshold_ssim):
    for original in test_images:
        original = np.expand_dims(original, axis=-1)  # Add channel dimension
        original = np.expand_dims(original, axis=0)   # Add batch dimension

        # Reconstruct the image
        reconstructed = vae.predict(original)

        # Compute SSIM map and score
        ssim_map, ssim_score = compute_ssim_map(np.squeeze(original), np.squeeze(reconstructed))

        # Highlight anomalous regions
        anomaly_mask = highlight_anomalous_regions(ssim_map, threshold=threshold_ssim)

        # Visualize the anomalies
        visualize_anomalous_regions(np.squeeze(original), np.squeeze(reconstructed), anomaly_mask)

# Example usage
test_image_paths = [
    'test/processed_image_edit.png', 
    r"C:\Users\priya\Documents\DL project\test 2\processed_image_edit.png",
    r"C:\Users\priya\Documents\DL project\test 3\processed_image_edit.png"
]  # Replace with actual image paths

# Load test images
test_images = [load_image(path) for path in test_image_paths]

# Load the saved VAE model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))
vae.load_weights("vae_model_epoch_15000.h5")  # Replace with the correct path

# Set SSIM threshold for anomaly detection
ssim_threshold = 0.85  # Adjust this threshold based on your use case

# Evaluate the model and display anomalies
evaluate_and_display_anomalies(vae, test_images, threshold_ssim=ssim_threshold)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, Model
from tensorflow.keras.models import Model
import albumentations as A
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.losses import MeanSquaredError
import tensorflow.keras.backend as K
import numpy as np

# Function to compute SSIM and KL Divergence based loss
def weighted_vae_loss(y_true, y_pred, mu, log_var, weight_ssim=0.8):
    mse_loss = MeanSquaredError()(y_true, y_pred)
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    kl_loss = -0.5 * tf.reduce_mean(1 + log_var - K.square(mu) - K.exp(log_var))
    loss = weight_ssim * ssim_loss + (1 - weight_ssim) * mse_loss + kl_loss
    return loss

# VAE model definition
def build_vae(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 256x256
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 128x128
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 64x64

    x = layers.Flatten()(x)
    mu = layers.Dense(128)(x)
    log_var = layers.Dense(128)(x)

    def sampling(args):
        mu, log_var = args
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mu + tf.exp(0.5 * log_var) * epsilon

    z = layers.Lambda(sampling)([mu, log_var])

    # Decoder
    decoder_input = layers.Input(shape=(128,))
    x = layers.Dense(64 * 64 * 128, activation='relu')(decoder_input)
    x = layers.Reshape((64, 64, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 128x128
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 256x256
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 512x512

    outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

    encoder = Model(inputs, [z, mu, log_var], name='encoder')
    decoder = Model(decoder_input, outputs, name='decoder')

    vae_outputs = decoder(encoder(inputs)[0])
    vae = Model(inputs, vae_outputs, name='vae')

    vae.add_loss(weighted_vae_loss(inputs, vae_outputs, encoder(inputs)[1], encoder(inputs)[2]))
    vae.compile(optimizer='adam')

    return vae, encoder, decoder

# Data augmentation using albumentations
def augment_image(image):
    # Convert to float32 for compatibility with augmentation operations
    image = image.astype(np.float32)

    transform = A.Compose([
        A.AdvancedBlur(p=0.5),
        A.CLAHE(p=0.5),
        A.Downscale(p=0.5),
        A.Emboss(p=0.5),
        A.Equalize(p=0.5),
        A.FancyPCA(p=0.5),
        A.GaussNoise(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CoarseDropout(p=0.5),
        A.PixelDropout(p=0.5)
    ])
    
    augmented = transform(image=image)
    return augmented['image']


# Simulate training data (assuming a single 512x512 clean brain image)
img_path = '61_processed_image_edit.png'
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Assuming grayscale MRI image
image = cv2.resize(image, (512, 512))  # Resize to (512, 512)

# Albumentations requires the input to have 3 channels, so expand dimensions
image = np.expand_dims(image, axis=-1)
image = image / 255.0  # Normalize

# Build VAE model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))

# Train the VAE
epochs = 5000
batch_size = 32

for epoch in range(epochs + 1):
    # Perform data augmentation on each epoch
    augmented_image = augment_image(image)
    augmented_image = np.expand_dims(augmented_image, axis=0)  # Add batch dimension

    # Train the model
    vae.fit(augmented_image, augmented_image, epochs=1, batch_size=batch_size, verbose=1)

    # Print and display outputs every 50 epochs
    if epoch % 50 == 0:
        reconstructed = vae.predict(np.expand_dims(image, axis=0))
        reconstructed = np.squeeze(reconstructed)  # Remove the batch dimension

        print(f"Epoch: {epoch}")
        
        # Display original, augmented, and reconstructed images
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))

        ax[0].imshow(np.squeeze(image), cmap='gray')
        ax[0].set_title("Original Image")
        ax[0].axis("off")

        ax[1].imshow(np.squeeze(augmented_image), cmap='gray')
        ax[1].set_title("Augmented Image")
        ax[1].axis("off")

        ax[2].imshow(reconstructed, cmap='gray')
        ax[2].set_title("Reconstructed Image")
        ax[2].axis("off")

        plt.show()

    # Save model every 100 epochs
    if epoch % 100 == 0:
        vae.save(f"vae_model_epoch_{epoch}.h5")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Define the same VAE architecture used for saving the model
def build_vae(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 256x256
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 128x128
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 64x64

    x = layers.Flatten()(x)
    mu = layers.Dense(128)(x)
    log_var = layers.Dense(128)(x)

    def sampling(args):
        mu, log_var = args
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mu + tf.exp(0.5 * log_var) * epsilon

    z = layers.Lambda(sampling)([mu, log_var])

    # Decoder
    decoder_input = layers.Input(shape=(128,))
    x = layers.Dense(64 * 64 * 128, activation='relu')(decoder_input)
    x = layers.Reshape((64, 64, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 128x128
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 256x256
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 512x512

    outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

    encoder = Model(inputs, [z, mu, log_var], name='encoder')
    decoder = Model(decoder_input, outputs, name='decoder')

    vae_outputs = decoder(encoder(inputs)[0])
    vae = Model(inputs, vae_outputs, name='vae')

    return vae, encoder, decoder

# Load the model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))
vae.load_weights("vae_model_epoch_5000.h5")  # Replace with the correct model path


In [ ]:
# Load and preprocess the anomalous image in the same way as the clean image

anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)


# Predict and compute anomaly
reconstructed_anomalous = vae.predict(anomalous_image)
difference = np.abs(anomalous_image - reconstructed_anomalous)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.subplot(1, 3, 2)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.subplot(1, 3, 3)
plt.title("Tumor Region (Difference)")
plt.imshow(np.squeeze(difference), cmap='hot')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load and preprocess the anomalous image in the same way as the clean image
anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)

# Predict and compute anomaly
reconstructed_anomalous = vae.predict(anomalous_image)
difference = np.abs(anomalous_image - reconstructed_anomalous)

# Thresholding to create a binary mask
threshold_value = 0.1  # Set a threshold value to identify anomaly
binary_mask = difference > threshold_value  # Binary mask (True where difference > threshold)

# Convert the mask to float for visualization purposes
binary_mask = binary_mask.astype(np.float32)

# Plot the anomalous image, reconstructed clean image, and the tumor region (binary mask)
plt.figure(figsize=(12, 4))

# Anomalous Image
plt.subplot(1, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.axis('off')

# Reconstructed Clean Image
plt.subplot(1, 3, 2)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.axis('off')

# Tumor Region (Binary Mask)
plt.subplot(1, 3, 3)
plt.title("Tumor Region (Binary Mask)")
plt.imshow(np.squeeze(binary_mask), cmap='hot')
plt.axis('off')

plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Load and preprocess the anomalous image in the same way as the clean image
anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)

# Pass the image through the encoder to get the latent space (z, mu, log_var)
z, mu, log_var = encoder.predict(anomalous_image)

# Decode the latent representation (z) to reconstruct the clean image
reconstructed_anomalous = decoder.predict(z)

# Compute the difference between the original anomalous image and the reconstructed image
difference = np.abs(anomalous_image - reconstructed_anomalous)

# Thresholding to create a binary mask for the tumor region
threshold_value = 0.2  # You can adjust this value
binary_mask = difference > threshold_value
binary_mask = binary_mask.astype(np.float32)

# Visualize the original anomalous image, reconstructed clean image, and the tumor region (binary mask)
plt.figure(figsize=(12, 6))

# Anomalous Image
plt.subplot(2, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.axis('off')

# Latent Space (mu)
plt.subplot(2, 3, 2)
plt.title("Latent Space (mu)")
plt.plot(mu[0])  # Plot the latent space (mu) for visualization
plt.axis('on')

# Latent Space (log_var)
plt.subplot(2, 3, 3)
plt.title("Latent Space (log_var)")
plt.plot(log_var[0])  # Plot the latent space (log_var) for visualization
plt.axis('on')

# Reconstructed Clean Image
plt.subplot(2, 3, 4)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.axis('off')

# Tumor Region (Difference)
plt.subplot(2, 3, 5)
plt.title("Tumor Region (Difference)")
plt.imshow(np.squeeze(difference), cmap='hot')
plt.axis('off')

# Tumor Region (Binary Mask)
plt.subplot(2, 3, 6)
plt.title("Tumor Region (Binary Mask)")
plt.imshow(np.squeeze(binary_mask), cmap='hot')
plt.axis('off')

plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

# Function to calculate reconstruction error
def calculate_reconstruction_error(original, reconstructed):
    return np.mean(np.square(original - reconstructed), axis=(1, 2, 3))  # MSE error per image

# Function to classify as anomalous based on a threshold
def classify_anomalies(reconstruction_error, threshold):
    return reconstruction_error > threshold

# Load test images (assuming grayscale 512x512 MRI images)
def load_image(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (512, 512))
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = image / 255.0  # Normalize
    return np.expand_dims(image, axis=0)  # Add batch dimension

# Define a function to evaluate model performance and show reconstructed images
def evaluate_model_and_display(vae, test_images, true_labels, threshold):
    reconstruction_errors = []
    predictions = []
    
    for img in test_images:
        original = img
        reconstructed = vae.predict(img)  # Predict reconstructed image
        error = calculate_reconstruction_error(original, reconstructed)
        reconstruction_errors.append(error)
        predictions.append(classify_anomalies(error, threshold))
    
        # Display original and reconstructed images
        plt.figure(figsize=(10, 5))
        
        # Original image
        plt.subplot(1, 2, 1)
        plt.imshow(np.squeeze(original), cmap='gray')
        plt.title("Original Image")
        plt.axis("off")

        # Reconstructed image
        plt.subplot(1, 2, 2)
        plt.imshow(np.squeeze(reconstructed), cmap='gray')
        if classify_anomalies(error, threshold):
            plt.title("Reconstructed Image (Anomalous)")
        else:
            plt.title("Reconstructed Image (Normal)")
        plt.axis("off")
        
        plt.show()
    
    reconstruction_errors = np.array(reconstruction_errors)
    predictions = np.array(predictions)

    # Calculate evaluation metrics
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    roc_auc = roc_auc_score(true_labels, reconstruction_errors)

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"ROC-AUC: {roc_auc}")

    return predictions, reconstruction_errors

# Example usage
test_image_paths = ['test/processed_image_edit.png',r"C:\Users\priya\Documents\DL project\test 2\processed_image_edit.png",r"C:\Users\priya\Documents\DL project\test 3\processed_image_edit.png"]  # Replace with actual image paths
test_images = [load_image(path) for path in test_image_paths]
true_labels = [1, 0, 0]  # Ground truth labels (0 = normal, 1 = anomalous)

# Load the saved VAE model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))
vae.load_weights("vae_model_epoch_5000.h5")  # Replace with the correct path

# Set an anomaly threshold
anomaly_threshold = 0.05  # You can tune this based on your validation results

# Evaluate the model and display reconstructed images
predictions, reconstruction_errors = evaluate_model_and_display(vae, test_images, true_labels, anomaly_threshold)

# Plot the reconstruction errors
plt.figure(figsize=(10, 5))
plt.plot(reconstruction_errors, label="Reconstruction Errors")
plt.axhline(y=anomaly_threshold, color='r', linestyle='--', label="Anomaly Threshold")
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, Model
from tensorflow.keras.models import Model
import albumentations as A
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.losses import MeanSquaredError
import tensorflow.keras.backend as K
import numpy as np

# Function to compute SSIM and KL Divergence based loss
def weighted_vae_loss(y_true, y_pred, mu, log_var, weight_ssim=0.8):
    mse_loss = MeanSquaredError()(y_true, y_pred)
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    kl_loss = -0.5 * tf.reduce_mean(1 + log_var - K.square(mu) - K.exp(log_var))
    loss = weight_ssim * ssim_loss + (1 - weight_ssim) * mse_loss + kl_loss
    return loss

# VAE model definition
def build_vae(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 256x256
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 128x128
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 64x64

    x = layers.Flatten()(x)
    mu = layers.Dense(128)(x)
    log_var = layers.Dense(128)(x)

    def sampling(args):
        mu, log_var = args
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mu + tf.exp(0.5 * log_var) * epsilon

    z = layers.Lambda(sampling)([mu, log_var])

    # Decoder
    decoder_input = layers.Input(shape=(128,))
    x = layers.Dense(64 * 64 * 128, activation='relu')(decoder_input)
    x = layers.Reshape((64, 64, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 128x128
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 256x256
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 512x512

    outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

    encoder = Model(inputs, [z, mu, log_var], name='encoder')
    decoder = Model(decoder_input, outputs, name='decoder')

    vae_outputs = decoder(encoder(inputs)[0])
    vae = Model(inputs, vae_outputs, name='vae')

    vae.add_loss(weighted_vae_loss(inputs, vae_outputs, encoder(inputs)[1], encoder(inputs)[2]))
    vae.compile(optimizer='adam')

    return vae, encoder, decoder

# Data augmentation using albumentations
def augment_image(image):
    # Convert to float32 for compatibility with augmentation operations
    image = image.astype(np.float32)

    transform = A.Compose([
        A.AdvancedBlur(p=0.5),
        A.CLAHE(p=0.5),
        A.Downscale(p=0.5),
        A.Emboss(p=0.5),
        A.Equalize(p=0.5),
        A.FancyPCA(p=0.5),
        A.GaussNoise(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CoarseDropout(p=0.5),
        A.PixelDropout(p=0.5)
    ])
    
    augmented = transform(image=image)
    return augmented['image']


# Simulate training data (assuming a single 512x512 clean brain image)
img_path = '61_processed_image_edit.png'
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Assuming grayscale MRI image
image = cv2.resize(image, (512, 512))  # Resize to (512, 512)

# Albumentations requires the input to have 3 channels, so expand dimensions
image = np.expand_dims(image, axis=-1)
image = image / 255.0  # Normalize

# Build VAE model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))

# Train the VAE
epochs = 10000
batch_size = 32

# Modified inject_anomaly function to return both the augmented image and the distorted window
# Modified inject_anomaly function to place random-shaped windows on the original image
def inject_anomaly(image, window_size=200):
    h, w = image.shape[:2]
    # Select a random window area from the image
    x, y = np.random.randint(0, h - window_size), np.random.randint(0, w - window_size)
    anomaly_window = image[x:x+window_size, y:y+window_size]

    # Apply contrast change and elastic deformation
    anomaly_window = A.RandomBrightnessContrast(p=1.0, brightness_limit=(-.4,0.4), contrast_limit=(-0.6,0.6))(image=anomaly_window)["image"]
    anomaly_window = A.ElasticTransform(p=1.0, alpha=50, sigma=50)(image=anomaly_window)["image"]

    # Create a random mask (you can modify this to get different shapes)
    mask = np.zeros_like(anomaly_window, dtype=np.uint8)
    num_shapes = np.random.randint(1, 4)  # Number of shapes to create a random mask

    for _ in range(num_shapes):
        shape_type = np.random.choice(['ellipse', 'polygon'])
        if shape_type == 'ellipse':
            center = (np.random.randint(0, window_size), np.random.randint(0, window_size))
            axes = (np.random.randint(10, window_size // 2), np.random.randint(10, window_size // 2))
            angle = np.random.randint(0, 180)
            cv2.ellipse(mask, center, axes, angle, 0, 360, (255, 255, 255), -1)
        elif shape_type == 'polygon':
            num_points = np.random.randint(3, 7)
            points = np.array([[
                (np.random.randint(0, window_size), np.random.randint(0, window_size))
                for _ in range(num_points)
            ]], dtype=np.int32)
            cv2.fillPoly(mask, points, (255, 255, 255))

    # Apply mask to the anomaly window
    masked_anomaly = cv2.bitwise_and(anomaly_window, anomaly_window, mask=mask)

    # Place the modified, random-shaped window back into the original image
    anomaly_image = image.copy()
    new_x, new_y = np.random.randint(0, h - window_size), np.random.randint(0, w - window_size)

    # Insert the masked anomaly at the new location
    window_region = anomaly_image[new_x:new_x+window_size, new_y:new_y+window_size]
    np.copyto(window_region, masked_anomaly, where=mask.astype(bool))

    return anomaly_image, masked_anomaly



# Training loop with anomaly injection
epochs = 10000
batch_size = 32

for epoch in range(epochs + 1):
    # Inject new anomaly every 500 epochs
    if epoch % 500 == 0:
        # Create a new augmented image with anomalies and capture the anomaly window
        augmented_image, distorted_window = inject_anomaly(np.squeeze(image))
        augmented_image = np.expand_dims(augmented_image, axis=-1)  # Add channel dimension
        augmented_image = np.expand_dims(augmented_image, axis=0)  # Add batch dimension

    # Train the model with the augmented image containing anomalies
    # Ensure y (original image) has the correct batch and channel dimensions
    target_image = np.expand_dims(image, axis=-1)  # Add channel dimension
    target_image = np.expand_dims(target_image, axis=0)  # Add batch dimension
    
    # Train the model with the augmented image and the target image
    vae.fit(augmented_image, target_image, epochs=1, batch_size=batch_size, verbose=1)


    # Print and display outputs every 50 epochs
    if epoch % 50 == 0:
        reconstructed = vae.predict(np.expand_dims(image, axis=0))
        reconstructed = np.squeeze(reconstructed)  # Remove the batch dimension

        print(f"Epoch: {epoch}")
        
        # Display original, augmented, reconstructed images, and distorted window
        fig, ax = plt.subplots(1, 4, figsize=(20, 5))

        ax[0].imshow(np.squeeze(image), cmap='gray')
        ax[0].set_title("Original Image")
        ax[0].axis("off")

        ax[1].imshow(np.squeeze(augmented_image), cmap='gray')
        ax[1].set_title("Augmented with Anomaly")
        ax[1].axis("off")

        ax[2].imshow(reconstructed, cmap='gray')
        ax[2].set_title("Reconstructed Image")
        ax[2].axis("off")

        ax[3].imshow(distorted_window, cmap='gray')
        ax[3].set_title("Distorted Anomaly Window")
        ax[3].axis("off")

        plt.show()

    # Save the model every 100 epochs
    if epoch % 1000 == 0:
        vae.save(f"vae_model_epoch_{epoch}.h5")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Define the same VAE architecture used for saving the model
def build_vae(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 256x256
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 128x128
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)  # 64x64

    x = layers.Flatten()(x)
    mu = layers.Dense(128)(x)
    log_var = layers.Dense(128)(x)

    def sampling(args):
        mu, log_var = args
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return mu + tf.exp(0.5 * log_var) * epsilon

    z = layers.Lambda(sampling)([mu, log_var])

    # Decoder
    decoder_input = layers.Input(shape=(128,))
    x = layers.Dense(64 * 64 * 128, activation='relu')(decoder_input)
    x = layers.Reshape((64, 64, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 128x128
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 256x256
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 512x512

    outputs = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

    encoder = Model(inputs, [z, mu, log_var], name='encoder')
    decoder = Model(decoder_input, outputs, name='decoder')

    vae_outputs = decoder(encoder(inputs)[0])
    vae = Model(inputs, vae_outputs, name='vae')

    return vae, encoder, decoder

# Load the model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))
vae.load_weights("vae_model_epoch_10000.h5")  # Replace with the correct model path


In [ ]:
# Load and preprocess the anomalous image in the same way as the clean image

anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)


# Predict and compute anomaly
reconstructed_anomalous = vae.predict(anomalous_image)
difference = np.abs(anomalous_image - reconstructed_anomalous)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.subplot(1, 3, 2)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.subplot(1, 3, 3)
plt.title("Tumor Region (Difference)")
plt.imshow(np.squeeze(difference), cmap='hot')
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load and preprocess the anomalous image in the same way as the clean image
anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)

# Predict and compute anomaly
reconstructed_anomalous = vae.predict(anomalous_image)
difference = np.abs(anomalous_image - reconstructed_anomalous)

# Thresholding to create a binary mask
threshold_value = 0.1  # Set a threshold value to identify anomaly
binary_mask = difference > threshold_value  # Binary mask (True where difference > threshold)

# Convert the mask to float for visualization purposes
binary_mask = binary_mask.astype(np.float32)

# Plot the anomalous image, reconstructed clean image, and the tumor region (binary mask)
plt.figure(figsize=(12, 4))

# Anomalous Image
plt.subplot(1, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.axis('off')

# Reconstructed Clean Image
plt.subplot(1, 3, 2)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.axis('off')

# Tumor Region (Binary Mask)
plt.subplot(1, 3, 3)
plt.title("Tumor Region (Binary Mask)")
plt.imshow(np.squeeze(binary_mask), cmap='hot')
plt.axis('off')

plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Load and preprocess the anomalous image in the same way as the clean image
anomalous_img_path = 'test/processed_image_edit.png'
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)

# Pass the image through the encoder to get the latent space (z, mu, log_var)
z, mu, log_var = encoder.predict(anomalous_image)

# Decode the latent representation (z) to reconstruct the clean image
reconstructed_anomalous = decoder.predict(z)

# Compute the difference between the original anomalous image and the reconstructed image
difference = np.abs(anomalous_image - reconstructed_anomalous)

# Thresholding to create a binary mask for the tumor region
threshold_value = 0.2  # You can adjust this value
binary_mask = difference > threshold_value
binary_mask = binary_mask.astype(np.float32)

# Visualize the original anomalous image, reconstructed clean image, and the tumor region (binary mask)
plt.figure(figsize=(12, 6))

# Anomalous Image
plt.subplot(2, 3, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.axis('off')

# Latent Space (mu)
plt.subplot(2, 3, 2)
plt.title("Latent Space (mu)")
plt.plot(mu[0])  # Plot the latent space (mu) for visualization
plt.axis('on')

# Latent Space (log_var)
plt.subplot(2, 3, 3)
plt.title("Latent Space (log_var)")
plt.plot(log_var[0])  # Plot the latent space (log_var) for visualization
plt.axis('on')

# Reconstructed Clean Image
plt.subplot(2, 3, 4)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.axis('off')

# Tumor Region (Difference)
plt.subplot(2, 3, 5)
plt.title("Tumor Region (Difference)")
plt.imshow(np.squeeze(difference), cmap='hot')
plt.axis('off')

# Tumor Region (Binary Mask)
plt.subplot(2, 3, 6)
plt.title("Tumor Region (Binary Mask)")
plt.imshow(np.squeeze(binary_mask), cmap='hot')
plt.axis('off')

plt.tight_layout()
plt.show()



In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

# Function to calculate reconstruction error
def calculate_reconstruction_error(original, reconstructed):
    return np.mean(np.square(original - reconstructed), axis=(1, 2, 3))  # MSE error per image

# Function to classify as anomalous based on a threshold
def classify_anomalies(reconstruction_error, threshold):
    return reconstruction_error > threshold

# Load test images (assuming grayscale 512x512 MRI images)
def load_image(img_path):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (512, 512))
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = image / 255.0  # Normalize
    return np.expand_dims(image, axis=0)  # Add batch dimension

# Define a function to evaluate model performance and show reconstructed images
def evaluate_model_and_display(vae, test_images, true_labels, threshold):
    reconstruction_errors = []
    predictions = []
    
    for img in test_images:
        original = img
        reconstructed = vae.predict(img)  # Predict reconstructed image
        error = calculate_reconstruction_error(original, reconstructed)
        reconstruction_errors.append(error)
        predictions.append(classify_anomalies(error, threshold))
    
        # Display original and reconstructed images
        plt.figure(figsize=(10, 5))
        
        # Original image
        plt.subplot(1, 2, 1)
        plt.imshow(np.squeeze(original), cmap='gray')
        plt.title("Original Image")
        plt.axis("off")

        # Reconstructed image
        plt.subplot(1, 2, 2)
        plt.imshow(np.squeeze(reconstructed), cmap='gray')
        if classify_anomalies(error, threshold):
            plt.title("Reconstructed Image (Anomalous)")
        else:
            plt.title("Reconstructed Image (Normal)")
        plt.axis("off")
        
        plt.show()
    
    reconstruction_errors = np.array(reconstruction_errors)
    predictions = np.array(predictions)

    # Calculate evaluation metrics
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    roc_auc = roc_auc_score(true_labels, reconstruction_errors)

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print(f"ROC-AUC: {roc_auc}")

    return predictions, reconstruction_errors

# Example usage
test_image_paths = ['test/processed_image_edit.png',r"C:\Users\priya\Documents\DL project\test 2\processed_image_edit.png",r"C:\Users\priya\Documents\DL project\test 3\processed_image_edit.png"]  # Replace with actual image paths
test_images = [load_image(path) for path in test_image_paths]
true_labels = [1, 0, 0]  # Ground truth labels (0 = normal, 1 = anomalous)

# Load the saved VAE model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))
vae.load_weights("vae_model_epoch_5000.h5")  # Replace with the correct path

# Set an anomaly threshold
anomaly_threshold = 0.05  # You can tune this based on your validation results

# Evaluate the model and display reconstructed images
predictions, reconstruction_errors = evaluate_model_and_display(vae, test_images, true_labels, anomaly_threshold)

# Plot the reconstruction errors
plt.figure(figsize=(10, 5))
plt.plot(reconstruction_errors, label="Reconstruction Errors")
plt.axhline(y=anomaly_threshold, color='r', linestyle='--', label="Anomaly Threshold")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.metrics import structural_similarity as ssim

# Function to load an image
def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    image = cv2.resize(image, (512, 512))  # Resize to the required dimensions
    return image / 255.0  # Normalize to [0, 1]

# Function to compute SSIM map and score
def compute_ssim_map(original, reconstructed):
    ssim_map = ssim(original, reconstructed, full=True, data_range=1.0)[1]
    ssim_score = np.mean(ssim_map)  # Average SSIM score
    return ssim_map, ssim_score

# Function to highlight anomalous regions based on SSIM
def highlight_anomalous_regions(ssim_map, threshold):
    # Create the anomaly mask
    anomaly_mask = ssim_map < threshold
    return anomaly_mask.astype(float)  # Ensure the mask is in float format for visualization

# Function to visualize original, reconstructed, and anomaly mask
def visualize_anomalous_regions(original, reconstructed, anomaly_mask):
    plt.figure(figsize=(15, 5))

    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(original, cmap='gray')
    plt.title("Original Image")
    plt.axis("off")

    # Anomaly mask
    plt.subplot(1, 3, 2)
    plt.imshow(anomaly_mask, cmap='hot', alpha=0.6)  # Display anomalous regions using heatmap
    plt.title("Anomalous Regions (SSIM < Threshold)")
    plt.axis("off")

    # Reconstructed image
    plt.subplot(1, 3, 3)
    plt.imshow(reconstructed, cmap='gray')
    plt.title("Reconstructed Image")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# Function to evaluate model and display anomalies
def evaluate_and_display_anomalies(vae, test_images, threshold_ssim):
    for original in test_images:
        original = np.expand_dims(original, axis=-1)  # Add channel dimension
        original = np.expand_dims(original, axis=0)   # Add batch dimension

        # Reconstruct the image
        reconstructed = vae.predict(original)

        # Compute SSIM map and score
        ssim_map, ssim_score = compute_ssim_map(np.squeeze(original), np.squeeze(reconstructed))

        # Highlight anomalous regions
        anomaly_mask = highlight_anomalous_regions(ssim_map, threshold=threshold_ssim)

        # Visualize the anomalies
        visualize_anomalous_regions(np.squeeze(original), np.squeeze(reconstructed), anomaly_mask)

# Example usage
test_image_paths = [
    'test/processed_image_edit.png', 
    r"C:\Users\priya\Documents\DL project\test 2\processed_image_edit.png",
    r"C:\Users\priya\Documents\DL project\test 3\processed_image_edit.png"
]  # Replace with actual image paths

# Load test images
test_images = [load_image(path) for path in test_image_paths]

# Load the saved VAE model
vae, encoder, decoder = build_vae(input_shape=(512, 512, 1))
vae.load_weights("vae_model_epoch_5000.h5")  # Replace with the correct path

# Set SSIM threshold for anomaly detection
ssim_threshold = 0.85  # Adjust this threshold based on your use case

# Evaluate the model and display anomalies
evaluate_and_display_anomalies(vae, test_images, threshold_ssim=ssim_threshold)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load and preprocess the anomalous image in the same way as the clean image
anomalous_img_path = 'test/processed_image_edit.png'  # Path to the uploaded anomalous image
anomalous_image = cv2.imread(anomalous_img_path, cv2.IMREAD_GRAYSCALE)
anomalous_image = cv2.resize(anomalous_image, (512, 512))
anomalous_image = anomalous_image / 255.0
anomalous_image = np.expand_dims(np.expand_dims(anomalous_image, axis=0), axis=-1)

# Predict and compute anomaly using the pre-trained VAE model
reconstructed_anomalous = vae.predict(anomalous_image)
difference = np.abs(anomalous_image - reconstructed_anomalous)

# Thresholding to create a binary mask
threshold_value = 0.1  # Set a threshold value to identify anomaly
binary_mask = difference > threshold_value  # Binary mask (True where difference > threshold)

# Calculate the reconstruction error map (absolute difference)
error_map = np.squeeze(difference)  # Removing extra dimensions for visualization

# Normalize the error map for visualization
normalized_error_map = (error_map - np.min(error_map)) / (np.max(error_map) - np.min(error_map))

# Plot the anomalous image, reconstructed clean image, error heatmap, and the tumor region (binary mask)
plt.figure(figsize=(16, 5))

# Anomalous Image
plt.subplot(1, 4, 1)
plt.title("Anomalous Image")
plt.imshow(np.squeeze(anomalous_image), cmap='gray')
plt.axis('off')

# Reconstructed Clean Image
plt.subplot(1, 4, 2)
plt.title("Reconstructed Clean Image")
plt.imshow(np.squeeze(reconstructed_anomalous), cmap='gray')
plt.axis('off')

# Error Heatmap (Highlighting high reconstruction error regions)
plt.subplot(1, 4, 3)
plt.title("Reconstruction Error Heatmap")
plt.imshow(normalized_error_map, cmap='hot')  # 'hot' colormap for heatmap
plt.axis('off')

# Tumor Region (Binary Mask)
plt.subplot(1, 4, 4)
plt.title("Tumor Region (Binary Mask)")
plt.imshow(np.squeeze(binary_mask), cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import albumentations as A
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to compute SSIM loss
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))

# Build the GAN model
def build_gan(input_shape):
    # Generator model
    generator_input = layers.Input(shape=(100,))
    x = layers.Dense(64 * 64 * 128, activation='relu')(generator_input)
    x = layers.Reshape((64, 64, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 128x128
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 256x256
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 512x512

    generator_output = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

    generator = Model(generator_input, generator_output, name='generator')

    # Discriminator model
    discriminator_input = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(discriminator_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Flatten()(x)
    discriminator_output = layers.Dense(1, activation='sigmoid')(x)

    discriminator = Model(discriminator_input, discriminator_output, name='discriminator')

    # Compile the models
    discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return generator, discriminator

# Data augmentation using albumentations
def augment_image(image):
    # Convert to float32 for compatibility with augmentation operations
    image = image.astype(np.float32)

    transform = A.Compose([
        A.AdvancedBlur(p=0.5),
        A.CLAHE(p=0.5),
        A.Downscale(p=0.5),
        A.Emboss(p=0.5),
        A.Equalize(p=0.5),
        A.FancyPCA(p=0.5),
        A.GaussNoise(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CoarseDropout(p=0.5),
        A.PixelDropout(p=0.5)
    ])
    
    augmented = transform(image=image)
    return augmented['image']

# Simulate training data (assuming a single 512x512 clean brain image)
img_path = '61_processed_image_edit.png'
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Assuming grayscale MRI image
image = cv2.resize(image, (512, 512))  # Resize to (512, 512)

# Albumentations requires the input to have 3 channels, so expand dimensions
image = np.expand_dims(image, axis=-1)
image = image / 255.0  # Normalize

# Build GAN model
generator, discriminator = build_gan(input_shape=(512, 512, 1))

# Training loop with anomaly injection
epochs = 1000
batch_size = 32

# Modified inject_anomaly function to place random-shaped windows on the original image
def inject_anomaly(image, window_size=200):
    h, w = image.shape[:2]
    # Create a new image to avoid modifying the original
    anomaly_image = image.copy()
    
    for _ in range(32):  # Create 32 anomalies for the batch
        # Select a random window area from the image
        x, y = np.random.randint(0, h - window_size), np.random.randint(0, w - window_size)
        anomaly_window = anomaly_image[x:x + window_size, y:y + window_size]

        # Apply contrast change and elastic deformation
        anomaly_window = A.RandomBrightnessContrast(p=1.0, brightness_limit=(-.4, 0.4), contrast_limit=(-0.6, 0.6))(image=anomaly_window)["image"]
        anomaly_window = A.ElasticTransform(p=0.7, alpha=50, sigma=50)(image=anomaly_window)["image"]

        # Create a random mask
        mask = np.zeros_like(anomaly_window, dtype=np.uint8)
        num_shapes = np.random.randint(1, 4)  # Number of shapes to create a random mask

        for _ in range(num_shapes):
            shape_type = np.random.choice(['ellipse', 'polygon'])
            if shape_type == 'ellipse':
                center = (np.random.randint(0, window_size), np.random.randint(0, window_size))
                axes = (np.random.randint(10, window_size // 2), np.random.randint(10, window_size // 2))
                angle = np.random.randint(0, 180)
                cv2.ellipse(mask, center, axes, angle, 0, 360, (255, 255, 255), -1)
            elif shape_type == 'polygon':
                num_points = np.random.randint(3, 7)
                points = np.array([[
                    (np.random.randint(0, window_size), np.random.randint(0, window_size))
                    for _ in range(num_points)
                ]], dtype=np.int32)
                cv2.fillPoly(mask, points, (255, 255, 255))

        # Apply mask to the anomaly window
        masked_anomaly = cv2.bitwise_and(anomaly_window, anomaly_window, mask=mask)

        # Place the modified, random-shaped window back into the original image
        window_region = anomaly_image[x:x + window_size, y:y + window_size]
        np.copyto(window_region, masked_anomaly, where=mask.astype(bool))

    return anomaly_image  # Return the entire batch of anomalies

# Training the GAN
for epoch in range(epochs + 1):
    # Inject new anomalies every 500 epochs
    if epoch % 500 == 0:
        # Create a new augmented image with anomalies
        # Create a new augmented image with anomalies
        augmented_image = inject_anomaly(np.squeeze(image))  # Now it's the entire image with anomalies
        
        # Expand dimensions and replicate the augmented image to create a batch of size 32
        augmented_images = np.expand_dims(augmented_image, axis=0)
        augmented_images = np.repeat(augmented_images, batch_size, axis=0)  # Shape should be (32, 512, 512, 1)


    # Generate random noise for generator input
    noise = np.random.normal(0, 1, (batch_size, 100))
    
    # Generate images using the generator
    generated_images = generator.predict(noise)

    # Prepare real and fake labels
    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    # Train discriminator
    discriminator_loss_real = discriminator.train_on_batch(augmented_images, real_labels)
    discriminator_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    generator_loss = discriminator.train_on_batch(generator.predict(noise), real_labels)

    # Print and display outputs every 50 epochs
    if epoch % 50 == 0:
        reconstructed = generator.predict(noise)
        reconstructed = np.squeeze(reconstructed)  # Remove the batch dimension

        print(f"Epoch: {epoch}, Discriminator Loss: {discriminator_loss[0]}, Generator Loss: {generator_loss}")

        # Display original, augmented, reconstructed images
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))

        ax[0].imshow(np.squeeze(image), cmap='gray')
        ax[0].set_title("Original Image")
        ax[0].axis("off")

        ax[1].imshow(np.squeeze(augmented_image), cmap='gray')
        ax[1].set_title("Augmented with Anomaly")
        ax[1].axis("off")

        ax[2].imshow(reconstructed[0], cmap='gray')
        ax[2].set_title("Generated Image")
        ax[2].axis("off")

        plt.show()

    # Save the models every 1000 epochs
    if epoch % 1000 == 0:
        generator.save(f"generator_epoch_{epoch}.h5")
        discriminator.save(f"discriminator_epoch_{epoch}.h5")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import albumentations as A
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to compute SSIM loss
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))

# Build the GAN model
def build_gan(input_shape):
    # Generator model
    generator_input = layers.Input(shape=(100,))
    x = layers.Dense(64 * 64 * 128, activation='relu')(generator_input)
    x = layers.Reshape((64, 64, 128))(x)
    x = layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 128x128
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 256x256
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)  # 512x512

    generator_output = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)
    generator = Model(generator_input, generator_output, name='generator')

    # Discriminator model
    discriminator_input = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(discriminator_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    discriminator_output = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(discriminator_input, discriminator_output, name='discriminator')

    # Compile the discriminator with a smaller learning rate
    discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                          loss='binary_crossentropy', metrics=['accuracy'])
    
    return generator, discriminator

# Data augmentation using albumentations
def augment_image(image):
    transform = A.Compose([
        A.AdvancedBlur(p=0.5),
        A.CLAHE(p=0.5),
        A.Downscale(p=0.5),
        A.Emboss(p=0.5),
        A.Equalize(p=0.5),
        A.FancyPCA(p=0.5),
        A.GaussNoise(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CoarseDropout(p=0.5),
        A.PixelDropout(p=0.5)
    ])
    augmented = transform(image=image)
    return augmented['image']

# Simulate training data (assuming a single 512x512 clean brain image)
img_path = '61_processed_image_edit.png'
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Assuming grayscale MRI image
image = cv2.resize(image, (512, 512))  # Resize to (512, 512)
image = np.expand_dims(image, axis=-1) / 255.0  # Normalize

# Build GAN model
generator, discriminator = build_gan(input_shape=(512, 512, 1))

# Training loop with anomaly injection
epochs = 25000
batch_size = 32

# Modified inject_anomaly function to place random-shaped windows on the original image
def inject_anomaly(image, window_size=200):
    h, w = image.shape[:2]
    anomaly_image = image.copy()
    
    for _ in range(32):  # Create 32 anomalies for the batch
        x, y = np.random.randint(0, h - window_size), np.random.randint(0, w - window_size)
        anomaly_window = anomaly_image[x:x + window_size, y:y + window_size]
        anomaly_window = A.RandomBrightnessContrast(p=1.0, brightness_limit=(-.4, 0.4), contrast_limit=(-0.6, 0.6))(image=anomaly_window)["image"]
        anomaly_window = A.ElasticTransform(p=0.7, alpha=50, sigma=50)(image=anomaly_window)["image"]
        
        mask = np.zeros_like(anomaly_window, dtype=np.uint8)
        num_shapes = np.random.randint(1, 4)
        
        for _ in range(num_shapes):
            shape_type = np.random.choice(['ellipse', 'polygon'])
            if shape_type == 'ellipse':
                center = (np.random.randint(0, window_size), np.random.randint(0, window_size))
                axes = (np.random.randint(10, window_size // 2), np.random.randint(10, window_size // 2))
                angle = np.random.randint(0, 180)
                cv2.ellipse(mask, center, axes, angle, 0, 360, (255, 255, 255), -1)
            elif shape_type == 'polygon':
                points = np.array([[ (np.random.randint(0, window_size), np.random.randint(0, window_size)) for _ in range(np.random.randint(3, 7))]], dtype=np.int32)
                cv2.fillPoly(mask, points, (255, 255, 255))
        
        masked_anomaly = cv2.bitwise_and(anomaly_window, anomaly_window, mask=mask)
        window_region = anomaly_image[x:x + window_size, y:y + window_size]
        np.copyto(window_region, masked_anomaly, where=mask.astype(bool))
    
    return anomaly_image

# Training the GAN
for epoch in range(epochs + 1):
    if epoch % 500 == 0:
        augmented_image = inject_anomaly(np.squeeze(image))
        augmented_images = np.expand_dims(augmented_image, axis=0)
        augmented_images = np.repeat(augmented_images, batch_size, axis=0)
        augmented_images += 0.05 * np.random.normal(0, 1, augmented_images.shape)

    noise = np.random.normal(0, 1, (batch_size, 100))
    generated_images = generator.predict(noise)
    generated_images += 0.05 * np.random.normal(0, 1, generated_images.shape)

    real_labels = 0.9 * np.ones((batch_size, 1))  # Label smoothing
    fake_labels = np.zeros((batch_size, 1))

    discriminator_loss_real = discriminator.train_on_batch(augmented_images, real_labels)
    discriminator_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

    for _ in range(2):  # Update generator twice
        noise = np.random.normal(0, 1, (batch_size, 100))
        generator_loss = discriminator.train_on_batch(generator.predict(noise), real_labels)

    if epoch % 50 == 0:
        reconstructed = generator.predict(noise)
        reconstructed = np.squeeze(reconstructed)

        print(f"Epoch: {epoch}, Discriminator Loss: {discriminator_loss[0]}, Generator Loss: {generator_loss}")

        fig, ax = plt.subplots(1, 3, figsize=(15, 5))
        ax[0].imshow(np.squeeze(image), cmap='gray')
        ax[0].set_title("Original Image")
        ax[0].axis("off")
        ax[1].imshow(np.squeeze(augmented_image), cmap='gray')
        ax[1].set_title("Augmented with Anomaly")
        ax[1].axis("off")
        ax[2].imshow(reconstructed[0], cmap='gray')
        ax[2].set_title("Generated Image")
        ax[2].axis("off")
        plt.show()

    if epoch % 1000 == 0:
        generator.save(f"generator_epoch_{epoch}.h5")
        discriminator.save(f"discriminator_epoch_{epoch}.h5")
